In [ ]:
%pip install --quiet --upgrade langchain langchain-community langchain-chroma langchain_huggingface datasets bitsandbytes langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:

In [ ]:
# import os
# import huggingface_hub
# from google.colab import userdata

# hf_token = os.environ.get("HF_TOKEN")

# # Use the token with the Hugging Face library
# huggingface_hub.login(token=hf_token)

# if "OPENAI_API_KEY" not in os.environ:
#     os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
from huggingface_hub import login

login(
  token="FILL", # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PATH_TO_DRIVE = "/content/drive/MyDrive/Capstone-local"

# Retrieval

In [ ]:
import os
import chromadb
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from langchain_chroma import Chroma
from tqdm import tqdm

# Load the dataset
ds = load_dataset("Studeni/AMAZON-Products-2023", split="train")
# Limit dataset to the first 1,000 entries
ds_limited = ds.select(range(1000))


# Extract relevant text (assuming 'title' and 'description' are important fields)
documents = []
for row in tqdm(ds_limited):
    if row['title'] and row['description']:
        documents.append(f"{row['title']} - {row['description']}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.14k [00:00<?, ?B/s]

data-00000-of-00004.arrow:   0%|          | 0.00/437M [00:00<?, ?B/s]

data-00001-of-00004.arrow:   0%|          | 0.00/432M [00:00<?, ?B/s]

data-00002-of-00004.arrow:   0%|          | 0.00/422M [00:00<?, ?B/s]

data-00003-of-00004.arrow:   0%|          | 0.00/423M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/117243 [00:00<?, ? examples/s]

100%|██████████| 1000/1000 [00:00<00:00, 1209.02it/s]


In [ ]:
documents[0]

"Anomie & Bonhomie - Amazon.com\nFans of Scritti Politti's synth-pop-funk masterpiece\nCupid & Psyche 85\nmay be shocked by how far afield Scritti mastermind Green Gartside has gone since then.\nAnomie & Bonhomie\n, his return to recording after a decadelong absence, ranges from guest shots by rappers and funksters such as\nMos Def\nand\nMe'Shell Ndegeocello\nto\nFoo Fighters\ntributes. Gartside's trademark breathy vocals and spot-on melodicism do find their places here, but are often forced to make way for other influences. Neither a total success nor a total failure,\nAnomie\ndoes display a spark that makes one hope that Gartside doesn't wait so long to record again.\n--Rickey Wright"

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

# Load SentenceTransformer model to generate embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

<ipython-input-11-b51055ffe7bc>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings

documents = [
    Document(page_content=doc, metadata={"topic": f"Product {i}"})
    for i, doc in enumerate(documents)
]

# ✅ Store documents in ChromaDB
db = Chroma.from_documents(
    documents=documents,
    embedding=embedding_function,
    persist_directory="db"
)

print("✅ Documents and embeddings stored successfully in ChromaDB!")

✅ Documents and embeddings stored successfully in ChromaDB!


In [ ]:
retriever = db.as_retriever(search_kwargs={"k": 5})

query = "What are some product categories?"
retrieved_docs = retriever.invoke(query)

print("Retrieved Docs:", retrieved_docs)

Retrieved Docs: [Document(metadata={'topic': 'Product 738'}, page_content='CP05 Cheese Serve Bathtub Caddy Tray, Easoger Expandable Bamboo Bath Tray Caddy with Wood Tablet and Book Rack Holder, Wine Glass Slot Bath Tub Table Caddy Gifted for Loved Ones (Tamboo) - cheese serve'), Document(metadata={'topic': 'Product 731'}, page_content='DIEWU Mini Piggy Figures Miniatures Toy, Home Decoration Resin Cake Toppers Christmas Birthday Gift Desk Bonsai Decoration - Mini Piggy Figures Miniatures Toy, Home Decoration Resin Cake Toppers Christmas Birthday Gift Desk Bonsai Decoration'), Document(metadata={'topic': 'Product 909'}, page_content='BBQ Net Tube,Rolling Grilling Basket BBQ Basket Rotisserie,Grill Tool with Removable Mesh Cover,Accessories Grilling Gifts for Men Father (11.8 inch *2) - BBQ Net Tube,Rolling Grilling Basket BBQ Basket Rotisserie,Grill Tool with Removable Mesh Cover,Accessories Grilling Gifts for Men Dad rolling grilling baskets for outdoor grilling grilling accessories'),

In [ ]:
# # SELECT DATA SIZE TO INDEX
# dataset_size = ["0-10", "0-100", "0-1000", "0-10000"]

# # Retrieve and generate using the relevant snippets of the blog.
# retriever = initialize_retriever(dataset_size[3], embeddings)

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [ ]:
# from langchain_huggingface import HuggingFacePipeline
# from transformers import BitsAndBytesConfig
# import torch

# # 4-bit Quantization Config for BitsAndBytes
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,  # Enable 4-bit quantization
#     bnb_4bit_quant_type="nf4",  # Use Normal Float-4 (NF4) quantization
#     bnb_4bit_compute_dtype=torch.bfloat16,  # Use bfloat16 for computation
#     bnb_4bit_use_double_quant=True,  # Use double quantization for memory efficiency
# )

In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
# from langchain_huggingface import HuggingFacePipeline
# import torch

# # Ensure CUDA is available
# assert torch.cuda.is_available(), "CUDA is not available. Ensure your GPU is detected."

# # Define Model ID
# model_id = "meta-llama/Llama-3.1-8B-Instruct"

# # 4-bit Quantization Config for BitsAndBytes
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,  # Enable 4-bit quantization
#     bnb_4bit_quant_type="nf4",  # Use Normal Float-4 (NF4) quantization
#     bnb_4bit_compute_dtype=torch.bfloat16,  # Use bfloat16 for computation efficiency
#     bnb_4bit_use_double_quant=True,  # Enable double quantization for memory savings
# )

# # Load Tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_id)

# # Ensure the tokenizer has a padding token
# if tokenizer.pad_token is None:
#     tokenizer.pad_token = tokenizer.eos_token  # Assign EOS token as padding
#     tokenizer.pad_token_id = tokenizer.eos_token_id  # Also set in model config

# # Load Model with 4-bit Quantization
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     quantization_config=quantization_config,  # Apply 4-bit quantization
#     device_map="cuda",  # Map model automatically to GPU
# )

# # Ensure consistency in model config
# model.config.pad_token_id = tokenizer.eos_token_id

# # Build LangChain-compatible Text Generation Pipeline
# llm = HuggingFacePipeline(
#     pipeline=pipeline(
#         "text-generation",
#         model=model,
#         tokenizer=tokenizer,
#         max_new_tokens=512,  # Limit output length
#         temperature=0.1,  # Lower temperature for more deterministic responses
#         do_sample=True,  # Enable sampling
#         top_p=0.9,  # Nucleus sampling
#         repetition_penalty=1.03,  # Reduce repetition in responses
#         return_full_text=False,  # Only return generated text
#     )
# )


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Device set to use cuda


In [ ]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = "FILL"

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
llm.invoke("What is the meaning of life?")

AIMessage(content='The meaning of life is a profound and philosophical question that has been explored by thinkers, theologians, and individuals throughout history. Different cultures, religions, and philosophies offer various interpretations:\n\n1. **Religious Perspectives**: Many religions provide specific meanings, often centered around the idea of serving a higher power, following moral guidelines, and seeking an afterlife or spiritual fulfillment.\n\n2. **Philosophical Views**: Philosophers have proposed numerous theories, from existentialism, which suggests that individuals create their own meaning through choices and actions, to utilitarianism, which focuses on maximizing happiness and reducing suffering.\n\n3. **Personal Meaning**: For many, the meaning of life is subjective and can be found in personal experiences, relationships, achievements, and the pursuit of passions and goals.\n\n4. **Scientific Perspectives**: Some may argue that life has no inherent meaning beyond biolo

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
retriever.invoke("With all the products you have seen, give different categories of the product and how they are related")

[Document(metadata={'topic': 'Product 703'}, page_content='Nutribolidos Keto Apple Cider Vinegar Gummies - ACV Gummies - Made in FDA Registered Facility - Immune Support - Digestive Health - keto apple cider vinegar gummies'),
 Document(metadata={'topic': 'Product 738'}, page_content='CP05 Cheese Serve Bathtub Caddy Tray, Easoger Expandable Bamboo Bath Tray Caddy with Wood Tablet and Book Rack Holder, Wine Glass Slot Bath Tub Table Caddy Gifted for Loved Ones (Tamboo) - cheese serve'),
 Document(metadata={'topic': 'Product 732'}, page_content='kivus Transform Keto+ Gummies - Transform Keto+ Apple Cider Vinegar Gummies (3 Pack, 180 Gummies) - Transform Keto+ Gummies - Transform Keto+ Apple Cider Vinegar Gummies (3 Pack, 180 Gummies)'),
 Document(metadata={'topic': 'Product 909'}, page_content='BBQ Net Tube,Rolling Grilling Basket BBQ Basket Rotisserie,Grill Tool with Removable Mesh Cover,Accessories Grilling Gifts for Men Father (11.8 inch *2) - BBQ Net Tube,Rolling Grilling Basket BBQ 

Generating Q-A Pairs

In [ ]:
from datasets import load_dataset

ds = load_dataset("Studeni/AMAZON-Products-2023", split="train")

In [ ]:
docs = list(ds['description'][:10])

In [ ]:
!pip install ragas

In [ ]:
import random
from langchain_core.prompts import ChatPromptTemplate

num_questions = 10
synth_questions = []
synth_answers = []

simple_prompt = ChatPromptTemplate.from_messages([('system', '{system}'), ('user', '{input}')])


for i in range(num_questions):
    doc1, doc2 = random.sample(docs, 2)
    sys_msg = (
        "Use the documents provided by the user to generate an interesting question-answer pair."
        " Try to use both documents if possible, and rely more on the document bodies than the summary."
        " Use the format:\nQuestion: (good question, 1-3 sentences, detailed)\n\nAnswer: (answer derived from the documents)"
    )
    usr_msg = (
        f"Document1: {(doc1)}\n\n"
        f"Document2: {(doc2)}"
        "Use the documents provided by the user to generate an interesting question-answer pair."
        " Try to use both documents if possible, and rely more on the document bodies than the summary."
        " Use the format:\nQuestion: (good question, 1-3 sentences, detailed)\n\nAnswer: (answer derived from the documents)"
    )

    qa_pair = (simple_prompt | llm).invoke({'system': sys_msg, 'input': usr_msg})
    synth_questions += [qa_pair.content.split('\n\n')[0]]
    synth_answers += [qa_pair.content.split('\n\n')[1]]
    print(f"QA Pair {i+1}")
    print(synth_questions[-1])
    print(synth_answers[-1])
    print()

QA Pair 1
Question: How do Adam Lambert's musical interpretations in 'High Drama' compare to the innovative style of the original Broadway cast album of 'Hamilton' in terms of genre blending and vocal performance?
Answer: Adam Lambert's 'High Drama' showcases a diverse collection of covers that re-imagine songs across various genres, including pop and rock, highlighting his unique vocal flair and emotional depth. In contrast, the original Broadway cast album of 'Hamilton' merges hip-hop with musical theater, creating a groundbreaking fusion that redefines storytelling through music. Both works exemplify innovative vocal performances, with Lambert's interpretations bringing a contemporary twist to established songs, while 'Hamilton' utilizes rap and melodic elements to narrate historical events, showcasing the versatility and evolution of modern music.

QA Pair 2
Question: How do the cultural expressions of the Garifuna people, as showcased in their music and dance, compare to the conte

In [ ]:
import pandas as pd

df_test = pd.DataFrame(zip(synth_questions,synth_answers),columns=['questions','ground_truth'])

df_test.to_csv(f'{PATH_TO_DRIVE}/generated_qa_pair_10.csv',index=False)

Generating Answers

In [ ]:
import pandas as pd
df_test = pd.read_csv(f'{PATH_TO_DRIVE}/generated_qa_pair_10.csv',encoding='ISO-8859-1')

In [ ]:
df_test

,questions,ground_truth
0,Question: How do Adam Lambert's musical interp...,Answer: Adam Lambert's 'High Drama' showcases ...
1,Question: How do the cultural expressions of t...,Answer: The Garifuna people's cultural express...
2,Question: How do the cultural expressions of t...,Answer: The Garinagu people's cultural express...
3,Question: How does Susan Boyle's second studio...,Answer: Susan Boyle's second studio album show...
4,Question: How does the cultural resilience of ...,Answer: The Garinagu people's cultural resilie...
5,Question: How does Adam Lambert's 'High Drama'...,Answer: Adam Lambert's 'High Drama' features a...
6,Question: How do the musical styles and themes...,"Answer: ATEEZ's ""SET: WITNESS,"" released on De..."
7,"Question: How did the evolution of the 12"" sin...","Answer: The 12"" single format, which emerged a..."
8,"Question: How did the introduction of the 12"" ...","Answer: The introduction of the 12"" single for..."
9,Question: How do the musical styles and themes...,"Answer: Bob Dylan's ""Time Out of Mind,"" with i..."


In [ ]:
from ragas import EvaluationDataset, evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import (
    LLMContextRecall, Faithfulness, FactualCorrectness,
    AnswerCorrectness, AnswerRelevancy, ContextRecall, ContextPrecision
)
import pandas as pd

# Generate responses & retrieve contexts using retriever.invoke()
responses = []
retrieved_contexts_list = []  # Store retrieved contexts for each question

for question in tqdm(df_test['questions']):
    print(f"\nProcessing: {question}")

    # Retrieve context using retriever
    retrieved_docs = retriever.invoke(question)

    # Extract context text from retrieved documents
    retrieved_contexts = [doc.page_content for doc in retrieved_docs if doc.page_content]

    if not retrieved_contexts:
        print("Warning: No relevant context retrieved!")

    # Generate response using the RAG pipeline
    response = rag_chain.invoke(question)

    # Store retrieved contexts and response
    responses.append(response)
    retrieved_contexts_list.append(retrieved_contexts)

  0%|          | 0/10 [00:00<?, ?it/s]


Processing: Question: How do Adam Lambert's musical interpretations in 'High Drama' compare to the innovative style of the original Broadway cast album of 'Hamilton' in terms of genre blending and vocal performance?


 10%|█         | 1/10 [00:01<00:14,  1.58s/it]


Processing: Question: How do the cultural expressions of the Garifuna people, as showcased in their music and dance, compare to the contemporary music scene represented by K-pop groups like ATEEZ, particularly in terms of community and tradition?


 20%|██        | 2/10 [00:07<00:34,  4.31s/it]

...

 80%|████████  | 8/10 [00:16<00:03,  1.69s/it]


Processing: Question: How did the introduction of the 12" single format in the 1970s influence the music industry, particularly in relation to the disco genre, and how does this compare to the impact of John Denver's music in the same era?


 90%|█████████ | 9/10 [00:18<00:01,  1.69s/it]


Processing: Question: How do the musical styles and themes of Bob Dylan's "Time Out of Mind" and the Broadway cast album of "Hamilton" reflect their respective cultural contexts and artistic intentions?


100%|██████████| 10/10 [00:19<00:00,  1.96s/it]


In [ ]:

# Construct dataset with retrieved contexts
dataset = [
    {
        "user_input": q,
        "ground_truth": gt,
        "retrieved_contexts": rc,  # Retrieved passages
        "response": r,
        "reference": gt  # Reference answer (typically same as ground truth)
    }
    for q, gt, r, rc in zip(df_test['questions'], df_test['ground_truth'], responses, retrieved_contexts_list)
]

# Convert to Ragas evaluation format
evaluation_dataset = EvaluationDataset.from_list(dataset)

In [ ]:
evaluation_dataset

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=10)

In [ ]:

# Define evaluator LLM
evaluator_llm = LangchainLLMWrapper(llm_gpt)
if evaluator_llm is None:
    raise ValueError("🚨 Evaluator LLM is not properly initialized!")

# Define metrics
metrics_list = [
    LLMContextRecall(),
    Faithfulness(),
    FactualCorrectness(),
    AnswerCorrectness(),
    AnswerRelevancy(),
    ContextRecall(),
    ContextPrecision()
]

# Run evaluation
result = evaluate(dataset=evaluation_dataset, metrics=metrics_list, llm=evaluator_llm)

# Display results
print("\n🔹 Evaluation Results:")
print(result)

Evaluating:   0%|          | 0/70 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[35]: BadRequestError(Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens. However, your messages resulted in 8202 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}})
ERROR:ragas.executor:Exception raised in Job[9]: APITimeoutError(Request timed out.)
ERROR:ragas.executor:Exception raised in Job[58]: APITimeoutError(Request timed out.)



🔹 Evaluation Results:
{'context_recall': 0.3833, 'faithfulness': 0.3854, 'factual_correctness(mode=f1)': 0.5525, 'answer_correctness': 0.7530, 'answer_relevancy': 0.9420, 'context_precision': 0.6000}


In [ ]:
df_test['rag_response'] = responses
df_test.to_csv('generated_qa_after_r_10000.csv',index=False)